In [ ]:
# create the simulation

library(MASS)
library(parallel)
library(tidyverse)
# library(arm)
# do a rep call
# library(glmnet)
# library(foreach)
# library(doParallel)
source("models.R")
source("sim.R")


In [ ]:
N_iter <- 500
n = 100
# num_preds <- c(500, 1000, 10000)
q <- 6

numCores <- detectCores() - 1

sims <- list()
for (p in c(500, 1000)) {
    sims[[p]] <- lapply(seq(1:N_iter), function(x) {
        sim.norm(n, p, q)
    })
    print(paste("done with", p))
}
sims[[10000]] <- lapply(seq(1:(N_iter/5)), function(x) {
    sim.norm(n, p = 10000, q)
})
print("done with 10000")


model_output <- mclapply(sims, function(x) {
    fit.models(x)
}, mc.cores = numCores)


df <- modelout_to_df(model_output)

df$p  <- c(rep(500, 500), rep(1000, 500), rep(10000, 100))
# save df to file as rda
saveRDS(df, "df.rda")

# group by model and summarise to get mean and standard deviation
summarydf <- df %>%
    group_by(p ,model) %>%
    summarise(
        FP_mean = mean(FP), FP_sd = sd(FP),
        FN_mean = mean(FN), FN_sd = sd(FN),
        auc_mean = mean(auc), auc_sd = sd(auc)
    )
# write to csv
write.csv(summarydf, "data/summarydf.csv")